# 通过微调提升大语言模型性能

---

到目前为止，你已经构建了一个具备记忆功能的聊天机器人，把它与真实世界的数据通过 RAG（检索增强生成）连接起来，并通过智能体赋予它调用外部工具的能力。迄今为止，我们主要依赖提示词和外部工具来引导模型的行为——但当模型本身缺少解决问题所需的基础能力时会发生什么？

想象一下，你的 AI 助手可以访问全世界的教科书，却仍然解不出一道基础的数学应用题。这正是我们面临的局限：再好的提示词或检索都无法弥补薄弱的推理能力。

这正是微调发挥作用的地方——它不是 RAG 或提示工程的替代品，而是强有力的补充。微调让我们能够“教会”模型如何思考、推理以及拆解复杂任务，使它从“消息灵通”变成真正“善于思考”。

## 到目前为止的故事……

用户们非常喜欢你的 AI 效率助手 **TaskFriend**！它会记住他们需要完成的事情，并结合真实世界的数据帮助他们提前规划。

但最近，用户发现一些不一致的情况。当被要求计算截止日期、估算工作量或解决简单数学题时，TaskFriend 经常在推理过程中半途而废，或给出错误答案。它知道公式，却难以逻辑地*应用*这些公式。

这不是提示词的问题——而是能力的缺口。就像一个读了课本却不懂数学的学生一样，**TaskFriend** 能检索信息，却无法有效推理。

我们需要一种方法来升级它的“思考引擎”——这正是微调所能实现的。

## 目标

* 理解模型微调的核心原理，包括损失函数、梯度下降和训练动态。
* 区分预训练和微调，并解释为什么微调是定制模型的最实用路径。
* 使用 LoRA（低秩适配）进行参数高效微调（PEFT），以降低内存和成本。
* 借助训练损失和验证损失诊断训练状态——欠拟合、过拟合以及收敛。
* 通过调整学习率、LoRA 秩和训练轮数等超参数迭代优化模型。
* 在基准任务上评估微调后的模型，并衡量性能提升。
* 通过动态加载 LoRA 或将其融合进全量权重的方式部署微调模型。

## 环境准备

模型微调需要大量硬件资源，尤其是 GPU 加速。为了完成本课程中的实验，我们建议在 **Platform for AI 的 Data Science Workshop（PAI-DSW）** 中创建一台 GPU 加速实例。

如果你无法访问 GPU 加速的环境，或者可用 GPU 的显存少于 30GB，我们强烈建议不要在本地运行本课程，因为在一些内存密集型实验中你可能会耗尽显存。

我们会继续使用 PAI-DSW，但在此之前，我们需要切换到另一台实例——一台**配备 GPU 的实例**。

> 如果你已经在使用 GPU 加速的 PAI-DSW 实例，可以直接继续使用。  
> 如果还没有，请参阅：`00 Setting Up the Environment`

完成上述步骤后，让我们从 [HuggingFace 模型库](https://huggingface.co/models) 下载模型：

In [ ]:
from huggingface_hub import snapshot_download

# Specify the repository ID of the model on Hugging Face Hub
repo_id = "Qwen/Qwen2.5-1.5B-Instruct"

# Specify the local directory where you want to save the model
local_dir = "./model/qwen2_5-1_5b-instruct"

# Download the model
snapshot_download(repo_id=repo_id, local_dir=local_dir)

# 你的任务：增强大语言模型的数学推理能力

提升数学问题求解能力一直是大语言模型发展的重点。要让你的智能助手发挥作用，它必须具备基础的计算与逻辑推理能力。

为了高效地开展模型微调，我们选择参数量较小的开源模型 **Qwen2.5-1.5B-Instruct** 作为基线模型，在性能、资源效率和可微调性之间取得平衡。

## 第一步：加载基线模型

首先，从 Hugging Face 下载模型并将其加载到内存中：

In [ ]:
%env TF_ENABLE_ONEDNN_OPTS=0

import torch
from swift.llm import (
    get_model_tokenizer, get_template, ModelType, get_default_template_type
)

# Get model information
model_type = ModelType.qwen2_5_1_5b_instruct
template_type = get_default_template_type(model_type)

# Configure local path of model
model_id_or_path = "./model/qwen2_5-1_5b-instruct"

# Intialize the model
kwargs = {}
model, tokenizer = get_model_tokenizer(model_type, torch.float32, model_id_or_path=model_id_or_path, model_kwargs={'device_map': 'auto'}, **kwargs)
model.generation_config.max_new_tokens = 128
template = get_template(template_type, tokenizer, default_system='')
print("✅ Model initialization complete!")

## 第二步：观察模型局限

我们将用一道简单的数学题来测试模型的表现。

> 农场主约翰拥有一块直角三角形的田地。如果田地的最长边是 15 米，并且每平方米可以收获 10 千克胡萝卜，那么从整块田地里总共可以收获多少千克胡萝卜？
> > **提示：** 答案是 `540kg`！

这里有一个关键点需要注意：我们设置了 `model.generation_config.max_new_tokens = 128`——我们希望模型在**尽量少的输出 token** 下解决问题，也就是追求**快速且高效的回答**。

In [ ]:
from swift.llm import inference
from IPython.display import Latex, display

math_question = """
    Farmer John has a right-angled triangular field. 
    If the longest edge of the field is 15 meters,
    and each square meter yields 10kg of carrots,
    what is the total weight of carrots harvested from the entire field?
    """
query = math_question
response, _ = inference(model, template, query)
print(query)
print("The answer we're looking for is: 540kg\n")
print("-" * 25 + "Model response" + "-" * 25)
display(Latex(response)) 
print("-" * 23 + "Model response end" + "-" * 23)

很明显，在限定输出 token 的情况下，这个模型依旧难以解出题目。只要放宽输出长度（你可以试试），它也许能解出来，但在限制下就会捉襟见肘。

使用检索增强生成（RAG）也会遇到类似的限制。我们之前提到过，RAG 更像是一次“开卷考试”——它可以增强信息获取能力，但并不会直接提升模型的推理能力。现实中，就像学生无法仅靠参考资料来弥补薄弱的数学基础一样，AI 模型也无法仅凭检索来弥补逻辑或数值推理的缺陷。

在这种情况下，如果想要直接提升问答助手在简单数学任务上的表现，**模型微调** 至关重要。通过训练一份精心准备的数学应用题及其推理步骤数据集，你可以强化模型的内部推理能力。

> **注意**：虽然微调能显著提升逻辑推理，但要保证**计算结果准确**（例如算术精度），仍然可以接入一个**计算器插件**或外部工具。这种方式能让模型把精确计算交给工具，从而实现“强推理 + 零误差计算”的组合。

总结一下，最优策略是：
- 通过 **微调** 提升模型的推理与任务拆解能力。
- 为模型配备 **计算器类工具**，保证算术结果可靠。

这套混合策略既能确保模型“想得明白”，也能保证结果“算得精准”。

# 模型微调原理（高能数学与理论预警！）

## 模型如何学习

### 机器学习：从数据中发现规律

在传统编程中，你通常会把明确的规则写进函数里。例如，一个简单的线性模型可以表示为：

$$
f(x) = ax
$$

这里 $ a $ 是已知且固定的参数（也称权重）。这个函数代表了一种基础算法模型，它将输入 $ x $ 映射为预测输出 $ y $。

然而，在真实世界的场景里，背后的规则（参数）往往是未知的。你拥有的是观测数据——一组组反映潜在模式的输入输出对。

**机器学习** 的目标，就是利用这些数据（训练集）来*学习*参数的最佳取值——这个过程被称为**模型训练**。我们不再手工编写规则，而是让模型通过示例自行推断。


### 衡量效果：损失函数与代价函数

要找到最优参数，你需要一种方式来衡量当前参数的表现。以模型 $ f(x) = ax $ 为例，我们来看看如何评估参数 $ a $。

#### 损失函数：单个样本的误差

对于每个训练样本 $ (x_i, y_i) $，你可以将模型预测 $ f(x_i) $ 与真实值 $ y_i $ 进行比较。它们之间的差异就是该样本的**损失**：

$$
L(y_i, f(x_i)) = y_i - ax_i
$$

不过，直接使用差值会带来一个问题：在求和时正负误差会相互抵消。为了解决这个问题，我们通常使用**平方误差**：

$$
L(y_i, f(x_i)) = (y_i - ax_i)^2
$$

平方可以确保所有误差都是正值，同时加大对大误差的惩罚，更容易识别表现不佳的参数组合。

> 在实际应用中，不同模型会使用不同形式的损失函数（例如分类问题常用交叉熵，鲁棒训练会选用 Huber 损失等）。

#### 代价函数：所有数据的平均误差

要评估模型在整个数据集上的表现，我们会对全部 $ m $ 个样本的损失求平均。这一整体指标被称为**代价函数**，也就是**均方误差（MSE）**：

$$
J(a) = \frac{1}{m} \sum_{i=1}^{m} (y_i - ax_i)^2
$$

代价函数量化了模型对整个训练数据的拟合程度。

> 在实践中，“损失函数”和“代价函数”这两个术语经常混用。在代码和工程语境中，“loss” 通常指一个批次的平均损失——本质上就是代价函数。

最小化代价函数就等价于找到让模型最佳拟合数据的参数 $ a $。从图形的角度看，这就像是在代价曲面上找到**最低点**。

<div style="text-align: center;">
  <img src="https://img.alicdn.com/imgextra/i4/O1CN0149XTTS1WUKSTtpeoh_!!6000000002791-2-tps-2314-1682.png" alt="Cost Function Curve" style="width: 400px;"/>
</div>




### 步步提升：梯度下降

**梯度下降** 是寻找函数最优参数最广泛使用的算法。它是一种优化方法，通过沿着梯度的反方向迭代更新，实现函数值的最小化。

想象一条 U 形曲线：你能找到**最低点（最小值）**吗？

> **提示：** 最小值就是曲线上最低的位置。

听上去很简单，对吧？

然而，对于拥有成千上万乃至上百万参数的机器学习模型而言，代价函数是一个高维曲面，我们既无法可视化，也无法用解析方法精确求解最小值。

**梯度下降** 算法的基本步骤如下：
1. 随机初始化参数。
2. 迭代计算梯度，并沿着能降低代价函数的方向更新参数。
3. 不断重复，直到收敛到最小值附近。

<div style="text-align: center; display: flex; justify-content: space-around; flex-wrap: wrap;">
  <img src="https://img.alicdn.com/imgextra/i2/O1CN01ihhR9Y1IbkFZTQ3bV_!!6000000000912-1-tps-1080-810.gif" alt="Gradient Descent 1D" style="width: 400px; margin: 5px;">
  <img src="https://img.alicdn.com/imgextra/i3/O1CN01meUISA1dHgq2mqm6V_!!6000000003711-1-tps-1080-810.gif" alt="Gradient Descent 2D" style="width: 400px; margin: 5px;">
</div>

要自动化这一过程，梯度下降需要解决两个关键问题：
* **方向**：参数应该往哪个方向调整？
* **步长**：每一步应该迈多大？



#### 顺着坡往下走：下降方向

回到那条 U 形代价曲线。最小值位于曲线的最低处——也就是斜率最平缓的位置。最佳移动方向，就是沿着斜率下降的方向，也就是**梯度的反方向**。

<div style="text-align: center;">
  <img src="https://img.alicdn.com/imgextra/i2/O1CN01ME3u6G203FVsQsmLe_!!6000000006793-2-tps-1608-1244.png" alt="Slope Direction" style="width: 400px;">
</div>

当我们进入更高的维度（例如三维曲面）时，**梯度** 是一个指向最陡峭上升方向的向量。因此，沿着梯度的**反方向**移动，可以最快降低函数值。

<div style="text-align: center;">
  <img src="https://img.alicdn.com/imgextra/i2/O1CN01Uh8OxI1mqnkBHqMjH_!!6000000005006-1-tps-664-684.gif" alt="3D Gradient" style="width: 400px;">
</div>

> - 对于二维曲线 $ f(a) $，梯度就是导数（斜率）。
> - 对于三维曲面 $ f(a,b) $，梯度是由偏导数组成的向量：$ \nabla f = \left( \frac{\partial f}{\partial a}, \frac{\partial f}{\partial b} \right) $，表示沿每个方向的变化率。



#### 学习速度：步长

搞清楚方向后，下一个挑战就是确定**步长**。

如果步长选择不当，最糟糕的情况就是算法会在最小值附近“跳来跳去”，永远无法收敛。

<div style="text-align: center;">
  <img src="https://img.alicdn.com/imgextra/i3/O1CN01y7FatQ27bKI9CYCJ1_!!6000000007815-1-tps-938-646.gif" alt="Fixed Step Oscillation" style="width: 400px;">
</div>

更好的做法是用**梯度的大小**来调整步长——梯度越小，步长越小，从而在最小值附近实现更精细的调整。

<div style="text-align: center">
<img src="https://img.alicdn.com/imgextra/i3/O1CN01h45Ifb1xRZhXXIXEC_!!6000000006440-1-tps-892-618.gif" style="width: 400px; display: block; margin-left: auto; margin-right: auto"/>
</div>

然而，当代价函数非常陡峭时，即便按照梯度缩放步长，也可能跨过最小值。为此，我们会引入一个缩放因子，也就是**学习率（α）**：

$$
\text{Step} = \alpha \cdot \nabla J(\theta)
$$

学习率对训练结果影响巨大：
- **太大**：可能越过最小值，导致发散。
- **太小**：收敛缓慢，耗时耗算力。
- **刚刚好**：能高效到达最优解。

<div style="display: flex; justify-content: space-between; gap: 15px; margin: 20px 0;">
  <div style="flex: 1; border: 1px solid #ddd; border-radius: 8px; padding: 15px; text-align: center;">
    <p style="margin: 0 0 10px;">学习率合适——收敛快且稳定。</p>
    <img src="https://img.alicdn.com/imgextra/i3/O1CN01NrvVfj1sCqtKHLyia_!!6000000005731-2-tps-1680-1224.png" alt="Good LR" style="width: 100%; border-radius: 5px;">
  </div>
  <div style="flex: 1; border: 1px solid #ddd; border-radius: 8px; padding: 15px; text-align: center;">
    <p style="margin: 0 0 10px;">学习率太低——收敛慢，成本高。</p>
    <img src="https://img.alicdn.com/imgextra/i1/O1CN015dbcz61MCn8LkN2Ta_!!6000000001399-2-tps-1728-1300.png" alt="Low LR" style="width: 100%; border-radius: 5px;">
  </div>
  <div style="flex: 1; border: 1px solid #ddd; border-radius: 8px; padding: 15px; text-align: center;">
    <p style="margin: 0 0 10px;">学习率太高——反复越界，难以收敛。</p>
    <img src="https://img.alicdn.com/imgextra/i1/O1CN01l4leTB1LKI0BcVs16_!!6000000001280-2-tps-1658-1262.png" alt="High LR" style="width: 100%; border-radius: 5px;">
  </div>
</div>

现代训练框架常会使用**自适应学习率策略**，比如线性衰减或余弦退火。以阿里云 PAI 的 **AutoML** 为例，它可以自动调节学习率，帮助模型更快找到最佳解。

### 关键训练超参数

#### 批量大小：一次处理多少样本？

每一次计算梯度并更新参数的过程称为一个**训练步（training step）**。

与其每次只用一个样本（随机梯度下降），或者一次性用全部数据（批量梯度下降），大多数训练会使用**小批量（mini-batch）**——即一次选取 $ n $ 个样本。

* **较大的批量**：训练更快，梯度更稳定，但需要更多显存。
* **较小的批量**：更新更噪声，但可能有助于泛化。

选择合适的批量大小需要在计算效率、收敛速度与模型表现之间权衡。

#### Eval Steps：多久检查一次进度？

由于数据集规模往往很大，我们不会在每个完整 epoch 后才去验证，而是在训练过程中隔一定步数就验证一次，也就是通过 `eval_steps` 控制验证频率，以便监测训练进度并及时发现过拟合。

#### Epochs：要复习多少遍？

一个 **epoch** 表示完整遍历整个训练集一次。

单个 epoch 往往不足以收敛，因此训练通常需要多个 epoch。但要注意：
- epoch 太少 → 欠拟合。
- epoch 太多 → 过拟合且浪费算力。

常见的做法是使用 **早停（Early Stopping）**：
- 让训练在预设的较大 epoch 数内持续进行。
- 同时监控验证集的表现。
- 当验证效果不再提升甚至变差时，提前停止训练。

另外一种策略是根据验证损失的趋势，动态调整学习率。

### 神经网络：AI 的“大脑”

#### 挑战在哪里

在文本生成等复杂任务中，输入与输出都具有高维特征，二者之间的映射关系也并不直观。我们该如何刻画这种复杂度？

数学上已经证明，**神经网络** 是一种**通用函数逼近器**——只要模型容量足够，它可以逼近任意连续函数。

<div style="text-align: center;">
  <img src="https://img.alicdn.com/imgextra/i1/O1CN01QRD5MH1rwMdJHBzxi_!!6000000005695-2-tps-1080-533.png" alt="Neural Network" style="width: 400px;">
</div>

单层网络可以表示为：

$$
Y = \sigma(W \cdot X)
$$

其中：
- $ X $：多维输入
- $ W $：权重矩阵（可学习参数）
- $ \sigma $：激活函数（引入非线性）
- $ Y $：输出

一个 $ k $ 层网络会将这些变换堆叠起来：

$$
Y = \sigma(W_k \cdots \sigma(W_2 \cdot \sigma(W_1 \cdot X)))
$$

#### 激活函数

激活函数用来决定一个神经元是否被激活以及激活程度。最常见的函数是 **ReLU**：

$$
\text{ReLU}(x) = \max(0, x) = 
\begin{cases}
x, & \text{if } x > 0 \\
0, & \text{if } x \leq 0
\end{cases}
$$

它能够引入非线性，使得网络可以表达更复杂的模式。

假设 $ X \in \mathbb{R}^{3\times2} $，$ W \in \mathbb{R}^{2\times3} $：

$$
\sigma(W_{2\times3} \cdot X_{3\times2}) = \sigma\left(
\begin{bmatrix}
w_{1,1} & w_{1,2} & w_{1,3} \\
w_{2,1} & w_{2,2} & w_{2,3}
\end{bmatrix}
\cdot
\begin{bmatrix}
x_{1,1} & x_{1,2} \\
x_{2,1} & x_{2,2} \\
x_{3,1} & x_{3,2}
\end{bmatrix}
\right)
$$

$$
= \begin{bmatrix}
\max(0, \sum_{k=1}^3 w_{1,k} x_{k,1}) & \max(0, \sum_{k=1}^3 w_{1,k} x_{k,2}) \\
\max(0, \sum_{k=1}^3 w_{2,k} x_{k,1}) & \max(0, \sum_{k=1}^3 w_{2,k} x_{k,2})
\end{bmatrix}
= Y_{2\times2}
$$

<div style="text-align: center;">
  <img src="https://img.alicdn.com/imgextra/i3/O1CN011caxP31GiUrEv1aGH_!!6000000000656-2-tps-847-779.png" alt="Neural Network Power" style="width: 400px;">
</div>

好消息是，即便在高维、非线性的空间中，**梯度下降依旧能够有效工作**。

## 高效微调技术

### 预训练 vs. 微调

在前面的章节中你已经了解到，模型训练本质上就是寻找最优参数集合。你在开篇下载的 `qwen2.5-1.5b-instruct` 模型已经拥有一套从海量通用数据中学习到的**预训练参数**。

**微调** 则是在此基础上再进一步。我们不是从头训练，而是用任务相关的数据（例如数学应用题）继续调整模型，让它在特定场景下表现更好。

那么，从零开始训练这样一个模型到底有多昂贵呢？我们来估算一下训练一个 15 亿参数模型所需的时间和硬件。

#### 硬件与显存需求

要微调大语言模型，通常需要高端 GPU，例如 **T4、A10、V100、A100 或 H100**：
- **T4、A10、V100**：在国内更容易获取且成本较低。
- **A100/H100**：性能更强，但价格高得多。

即便是 15 亿参数的模型，也需要相当可观的显存：

- 在全精度（FP32）下，每个参数占 **4 字节**。
- 参数本身所占内存约为  
  $$
  1.5 \times 10^9 \times 4 / 2^{30} \approx 5.59\,\text{GB}
  $$

但在训练过程中，还需要额外存储梯度、优化器状态和激活值。实际使用中，通常需要参数大小的 **7–8 倍显存**：

$$
5.59\,\text{GB} \times 8 \approx \textbf{45\,GB}
$$

这已经超出了大多数消费级 GPU 的显存——甚至超过了你当前实验环境的上限。

#### 估算训练时间与成本

假设你在阿里云租用一台 **8×V100 GPU 实例**：
- 总显存：256 GB（足够训练）
- 费用：约 35 美元/小时
- 处理速度：约 190 tokens/秒/卡

假设为了追求较高精度，你准备的数据量相当于**《指环王》三部曲全文复制 5,000 遍**（每本 455,000 字符），合计约 **22.8 亿 tokens**。

$$
\text{Time (days)} = \frac{2.28 \times 10^9}{8 \times 190 \times 3600 \times 24} \approx \textbf{17 天}
$$

17 天！再来看看成本：

$$
\text{Cost} = \left( \frac{2.28 \times 10^9}{8 \times 190 \times 3600} \right) \times 35 \approx \textbf{\$14,500}
$$

> 💡 作个对比：  
> Meta 的 **LLaMA-65B** 预训练使用了 **1.4 万亿 tokens**、**2,048 张 A100 GPU**，整整跑了 **21 天**。按每张 A100 每小时 3.93 美元估算，成本约 **400 万美元**。

这点明了一个核心结论：  
> **训练数据规模** 直接决定了所需的 **总算力**，从而影响训练的**时间**和**成本**。

虽说更好的硬件能缩短训练时间（有时还会降低总成本），但最有价值的杠杆是：
- **数据质量**：高质量、相关性强的数据能加快收敛。
- **高效训练方法**：减少需要更新的参数可以显著降低成本。

在现实中，大规模标注数据往往非常昂贵——尤其在医学诊断、法律文本等细分领域。

为此，现代 AI 采用了**两阶段策略**：

| 阶段 | 目标 | 数据类型 | 学习方式 |
|------|------|-----------|-----------------|
| **预训练** | 学习通用语言能力 | 海量未标注文本（网页、书籍、百科） | 自监督学习 |
| **微调** | 适配具体任务 | 小规模标注数据（如数学题、情感标签） | 监督学习 |

**二者的核心差异：**

<div style="overflow-x: auto; margin: 20px 0;">
  <table>
    <thead>
      <tr>
        <th>对比维度</th>
        <th>预训练</th>
        <th>微调</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td><strong>目标</strong></td>
        <td>学习通用模式与表征</td>
        <td>适配特定下游任务</td>
      </tr>
      <tr>
        <td><strong>数据规模</strong></td>
        <td>海量（TB 级文本）</td>
        <td>小规模（数千至数万样本）</td>
      </tr>
      <tr>
        <td><strong>数据标注</strong></td>
        <td>无需标注（自监督）</td>
        <td>需要标注（监督）</td>
      </tr>
      <tr>
        <td><strong>训练方式</strong></td>
        <td>自监督（如预测掩码词）</td>
        <td>监督（输入-输出配对）</td>
      </tr>
      <tr>
        <td><strong>参数更新</strong></td>
        <td>更新全部参数</td>
        <td>更新全部或部分参数</td>
      </tr>
      <tr>
        <td><strong>应用场景</strong></td>
        <td>构建基础模型（如 Qwen、GPT、DeepSeek）</td>
        <td>为特定业务定制模型</td>
      </tr>
    </tbody>
  </table>
</div>

通过这种两阶段策略，你可以：
1. **复用强大的预训练模型**（节省海量算力投入）。
2. **用少量标注数据完成微调**（节省时间与标注成本）。

**如何快速又省钱地定制模型**

只需遵循以下步骤，就能高效打造拥有领域专长的模型：

1. **选一个强大的预训练模型**，例如 Qwen2.5、DeepSeek 或 Llama。
2. **收集一份小而精的标注数据集**（例如 5,000 道附带解题步骤的数学应用题）。
3. **对模型进行微调**——速度快、成本低，远远优于从零训练。

不过，即便是微调，如果更新全部参数，成本也仍然昂贵。

那么：**我们能否进一步压缩 GPU 显存占用和训练成本？**

答案是肯定的——这就是**高效微调（Efficient Fine-Tuning）**。

#### 全量微调 vs. 参数高效微调（PEFT）

影响显存与成本的核心因素就是**需要更新的参数数量**。

我们可以把微调方式分成两大类：

* 🔹 全量微调
    - 更新模型的**全部参数**。
    - 效果强，但显存与算力需求高。
    - 即便使用预训练模型，对大模型来说依旧成本高昂。

* 🔹 参数高效微调（PEFT）
    - 只更新**一小部分参数**。
    - 显著降低显存、算力与成本。
    - 效果与全量微调接近。

常见的 PEFT 方法包括：
- **Adapter Tuning**：在网络层中插入小型神经模块。
- **Prompt Tuning**：学习“软提示词”，不改动模型权重。
- **LoRA（低秩适配）**：通过低秩矩阵更新权重变化，通常只需原参数的 **0.1%–1%**。

> ✅ **LoRA 已成为资源受限场景下的首选微调方法。**

接下来，我们将深入介绍 **LoRA 的工作原理**，看看它如何在极低开销下实现高性能微调。

### LoRA：低秩适配

**LoRA（Low-Rank Adaptation）** 已成为大模型高效微调中最广泛应用的方法。它适用于多种模型结构，能够在大幅减少可训练参数数量的同时保持优异性能。

#### LoRA 的工作原理

LoRA 并不会在微调过程中更新所有原始权重，而是通过一个巧妙的数学方法：把权重变化**分解为两个小型的低秩矩阵**。

核心公式如下：

$$
W_{\text{new}} = W_{\text{frozen}} + \Delta W = W_{\text{frozen}} + A \cdot B
$$

其中：
- $ W_{\text{frozen}} $：预训练得到的原始权重矩阵（被冻结，不参与训练）。
- $ \Delta W = A \cdot B $：需要学习的低秩更新。
- $ A \in \mathbb{R}^{d \times r} $, $ B \in \mathbb{R}^{r \times d} $：两个待训练的小矩阵。
- $ r \ll d $：秩远小于原始矩阵维度。

也就是说，我们只训练 $ A $ 和 $ B $，而不是整个 $ W $，因此显存和计算成本大幅下降。

#### 理解低秩分解

为了更好地理解它的原理，我们来看一个简单示例。

假设某一层的配置如下：
- 输入 $ X \in \mathbb{R}^4 $
- 输出 $ Y \in \mathbb{R}^5 $
- 权重矩阵 $ W \in \mathbb{R}^{5 \times 4} $ → 共 20 个参数

变换为：
$$
Y = \sigma(W \cdot X)
$$

微调时，我们不再重新训练 $ W $，而是学习一个小的增量 $ \Delta W $，假设它只包含**有限的新信息**，也就是具有**低秩**。

例如，下方的 $ 5 \times 4 $ 矩阵：

$$
\Delta W =
\begin{bmatrix}
1 & 0 & 2 & -1 \\
2 & 0 & 4 & -2 \\
3 & 0 & 6 & -3 \\
4 & 0 & 8 & -4 \\
5 & 0 & 10 & -5 \\
\end{bmatrix}
$$

尽管看起来庞大，但每一行都是第一行的倍数，因此它的**有效信息**可以由一个向量表示——矩阵的**秩为 1**。

我们可以把它分解为两个更小的矩阵：

$$
\Delta W = 
\begin{bmatrix} 1 \\ 2 \\ 3 \\ 4 \\ 5 \end{bmatrix}_{5 \times 1}
\cdot
\begin{bmatrix} 1 & 0 & 2 & -1 \end{bmatrix}_{1 \times 4}
= A \cdot B
$$

这样一来，我们只需训练 $ 5 + 4 = 9 $ 个参数，而不是 20 个，极大地节省了资源。

> ✅ 在真实模型中，研究者发现**微调所需的权重更新本身就是低秩的**——模型的大部分知识已经存在于预训练权重里，微调只需针对性地做少量调整。

#### LoRA 高效的秘密

我们将这个思路应用于实际使用的 `qwen2.5-1.5b-instruct` 模型。

假设：
- 层的维度 $ d = 1024 $
- LoRA 秩 $ r = 8 $（远小于原始维度，满足 $ r \ll d $）

| 微调方式 | 可训练参数 | 数量 | 节省比例 |
|-------|------------------------|-------|--------|
| 全量微调 | 整个 $$ W_{d \times d} $$ | $$ 1024 \times 1024 = 1,048,576 $$ | 0% |
| LoRA 微调 | $$ A_{d \times r} + B_{r \times d} $$ | $$ 1024 \times 8 + 8 \times 1024 = 16,384 $$ | **98.4%** |

✅ 只训练 **1.6% 的参数**，LoRA 的效果就能接近全量微调，同时显存、成本和时间大幅降低。


#### 推理阶段：为了速度而合并

在推理阶段，LoRA 权重可以合并回原模型：

$$
W_{\text{merged}} = W_{\text{original}} + A \cdot B
$$

合并方式有两种：
- **部署前合并**：生成一个独立的优化模型。
- **动态应用**：在多任务场景下按需加载 LoRA。

一旦合并完成，**推理阶段就不再有额外开销**，模型的运行体验与全量微调结果一致。

<div style="text-align: center;">
  <img src="images/LMP-C01_08-LoRA Matrix.jpg" style="max-width: 800px;" />
  <br>
  <small>LoRA 的工作方式</small>
</div>

**如何选择合适的秩（rank）**

秩 $ r $ 决定了适配器的容量：
- **较小的 $ r $** → 参数少、训练快、过拟合风险低。
- **较大的 $ r $** → 表达能力更强，但显存占用增加、可能过拟合。

#### 实践建议

| 训练数据规模 | 建议秩 $ r $ | 理由 |
|--------------------|--------------------------|---------|
| 小型（1k–10k 样本） | $ r \leq 16 $ | 避免过拟合，防止模型记住训练数据而不是学习模式 |
| 中型（10k–100k）   | $ r = 16 \text{ to } 32 $ | 在容量与效率之间取得平衡 |
| 大型（>100k 样本） | $ r \geq 32 $ | 能捕获更复杂的任务模式 |

> 💡 建议从 $ r = 8 $ 或 $ r = 16 $ 起步，再根据验证集表现做调整。


### LoRA 的表现

LoRA 原始论文评估了多种参数高效微调方法在两个基准数据集上的表现，结果一目了然：

<div style="text-align: center;">
  <img src="images/LMP-C01_09-VA vs Num Trainable Param.gif" style="max-width: 800px;" />
  <br>
  <small>性能与可训练参数数量的对比。<br>LoRA 以最少的参数赢得最高准确率。</small>
  <br>
  <small><i>来源：<a href="https://arxiv.org/abs/2106.09685" target="_blank">LoRA: Low-Rank Adaptation of Large Language Models</a></i></small>
</div>

#### 核心结论
- **可训练参数更多 ≠ 性能更好**。某些方法很快就达到瓶颈。
- **LoRA 的可扩展性极佳**：随着数据量或 rank 增加，性能仍能持续提升。
- **性价比最高**：LoRA 以极低开销换来接近全量微调的表现。

> ✅ 在实际工程中，无论是数学推理、代码生成还是垂直领域问答，LoRA 都是资源受限场景的首选微调方法。

通过把**预训练知识**与**低成本的定向适配**结合起来，LoRA 让高性能的模型定制变得既快速又经济，让先进的 AI 能力真正走向普及。

# 从论文到实战：微调我们的模型

---

## 模型训练状态与评估指标

训练一个机器学习模型就像学生备考。学生通过练习题、模拟考试以及正式考试来查漏补缺；同样，模型也通过三类不同的数据集来学习与评估，每个数据集都承担着独特的角色。

这些数据集帮助我们构建关键指标，反映模型的当前状态，并指导后续的训练策略。

### 模型的“三场考试”

| 数据集 | 作用 | 类比 | 核心指标 |
|--------|---------|--------|-----------|
| **训练集** | 通过调整参数来教模型 | 带答案的课后练习册 | **训练损失** |
| **验证集** | 在训练过程中评估学习进展 | 模拟考试 | **验证损失** |
| **测试集** | 训练完成后用于最终评估 | 正式考试（只考一次） | **准确率 / 测试得分** |

具体来看：

1. 训练集 → 训练损失
    - 相当于模型反复研读的“练习册”。
    - 在训练时，模型根据输入输出对来学习，并计算**训练损失**——衡量预测值与真实值之间的差距。
    - 训练损失下降，意味着模型在已见过的数据上表现变好。

> 🔁 模型会根据这些反馈（通过梯度下降）更新内部参数。

2. 验证集 → 验证损失
    - 把它当作**模拟考试**——训练中不会直接接触这部分数据。
    - 模型会定期在验证集上测试，以评估对新样本的泛化能力。
    - 得到的**验证损失**帮助我们提前发现过拟合或欠拟合。

> 📊 切记：不要用验证集来更新模型参数，它只用于监控性能。

3. 测试集 → 最终表现
    - 相当于**正式考试**，只在训练结束后使用一次。
    - 它能给出模型的最终、客观表现。
    - 评估完成后，不应再根据测试结果回头调整模型，否则评估就不再公平。

> ✅ 最佳实践：训练结束前，务必保持测试集“完全未被触碰”。

## 解析训练动态：三种关键状态

通过对比**训练损失**和**验证损失**，你可以判断模型目前所处的状态，并决定下一步怎么做。

| 训练损失 | 验证损失 | 状态 | 解释 | 应对策略 |
|---------------|-----------------|-------|----------------|------------|
| 不下降甚至上升 | 没有改善 | ❌ **训练失败** | 模型没有学会任何东西——就像学生完全没听懂课程内容。 | 检查学习率、数据质量或模型结构。 |
| 下降 | 下降 | ⏳ **欠拟合** | 模型还在学习中——练习册和模拟考都在进步，但远未达到理想水平。 | 继续训练；可以考虑增加 epoch。 |
| 下降 | 上升 | ⚠️ **过拟合** | 模型在“背答案”，而不是理解知识——练习题做得好，模拟考却越来越差。 | 尽早停止训练，增加正则化，或引入更多多样化数据。 |


### 读懂过拟合：“死记硬背”的陷阱

当模型**过拟合**时，它对训练数据过度依赖，就像学生只记住了答案，而没有掌握解题思路。

例如：
- 在“练习册”上的题目几乎都做对。
- 但遇到稍作变化的“模拟题”就束手无策。

**防止过拟合的常见做法：**
- 采用 **早停（Early Stopping）**：一旦验证损失不再下降就停止训练。
- 提升 **数据多样性**：增加更丰富的训练样本，避免死记硬背。
- 引入 **正则化** 技术：如 dropout、权重衰减等。
- **增大批量大小** 或使用 **数据增强**，降低学习过程中的噪声。

> 💡 小贴士：训练损失和验证损失之间有小幅差距是正常的。但如果训练损失持续下降、验证损失却开始上升，就该及时踩刹车了。

## 小结

想要成功微调模型，需要做到：
1. **全程监控训练损失与验证损失**。
2. **把验证集当作指南针**——而不是训练集。
3. **把测试集留到最后一次性评估**。
4. **针对训练失败、欠拟合、过拟合采取正确策略**。

就像一个好老师会关注学生在练习、模拟和正式考试中的表现一样，你也需要用恰当的指标在恰当的阶段引导你的模型。

如此一来，模型不仅能“记住答案”，还能真正学会**如何推理**。


# 微调实战

---

## 基线测试

在正式开始微调之前，先给模型来一份数学小测验：

In [ ]:
import json
from IPython.display import Markdown

sum, score = 0, 0
for line in open("./resources/benchmark_exam.jsonl"):

    math_question = json.loads(line)
    query = math_question["messages"][1]["content"]

    response, _ = inference(model, template, query)

    ans = math_question["messages"][2]["content"]
    pos = ans.find("ans")
    end_pos = ans[pos:].find('}}')
    ans = ans[pos - 2: end_pos + pos + 2]
    
    print("=" * 50)
    print(query.split("#Math Problem#\n")[1])
    print("The answer we're looking for is: " + ans + "\n")
    print("-" * 25 + "Model response" + "-" * 25)
    display(Latex(response)) 
    print("-" * 23 + "Model response end" + "-" * 23)

    if ans in response or ans[6:-2] in response:
        score += 1
        print("Model is correct!\n")
    else: print("Model is wrong :(\n")
    sum += 1

display(Markdown("Benchmark score: **" + str(int(100*score/sum)) + "**"))

基线模型在解题过程中经常半途放弃，得不到正确答案。这说明任务难度已经超出它当前的推理能力，也揭示了提示工程的根本局限：**提示词再优秀，也无法弥补底层能力的缺失**。

换句话说，提示工程就像给学生更好的考试说明——如果学生根本没学会知识，说明再清楚也无法提高成绩。

要真正提升模型在数学推理等任务上的表现，**必须进行模型微调**。通过训练相关示例，模型可以学会逐步推理的思考方式。

唯有微调，模型才能发展出可靠的逻辑和计算能力，从“模式匹配”走向真正的“问题求解”。

## 使用 `ms-swift` 进行微调

本节我们会使用 `**[MS-Swift](https://github.com/modelscope/ms-swift/tree/main)**`（ModelScope Scalable lightweight Infrastructure for Fine-Tuning）框架。它由阿里巴巴 ModelScope（MoE）团队开源，支持超过 **350 个大语言模型（LLM）** 和 **90+ 个多模态大模型（MLLM）** 的训练、推理、评估与部署，覆盖预训练、微调和对齐任务。

MS-Swift 的一大优势是**易用性**。在训练过程中，框架会自动：
- 定期在验证集上计算**验证损失**。
- 在每个验证步骤**保存模型检查点**。
- 在训练结束后保留**最佳模型检查点**（验证损失最低）。

<div style="text-align: center;">
  <img src="images/LMP-C01_08-Last Best Checkpoints.jpg" style="max-width: 800px; border: 1px solid white;" />
  <br>
  <small>ms-swift 在适用时会输出最新和最佳检查点</small>
</div>


## 实验设置

接下来的实验中，我们将重点调节三类超参数：
- **学习率 (`learning_rate`)**：控制优化过程中的步长。
- **LoRA 秩 (`lora_rank`)**：决定低秩适配的容量。
- **训练轮数 (`num_train_epochs`)**：模型完整遍历数据集的次数。

我们还会尝试不同的数据集，展示数据质量与规模对微调效果的影响。

其他参数（例如 `batch_size`、`max_length`）主要用于**缩短训练时间**并兼顾显存限制。本练习中无需对它们进行深入调参。



### 实验 1（≈1 分钟）

第一次运行建议采用以下配置。我们使用由 [**Qwen**](https://chat.qwen.ai/) 生成的 **100 道数学题及解答** 作为小数据集，便于快速迭代并与后续实验对比。

> **参数设置**
> 
> | 参数 | 数值 |
> |--------|-------|
> | **学习率** (`learning_rate`) | `0.1` |
> | **LoRA 秩** (`lora_rank`) | `4` |
> | **训练轮数** (`num_train_epochs`) | `1` |
> | **数据集路径** | `"./resources/training_dataset_100.jsonl"` |
> | **约束条件** | 
> | &nbsp;&nbsp;• `batch_size` ≤ 16 |（受显存限制）|
> | &nbsp;&nbsp;• `max_length` ≤ 512 |（单样本最大 token 长度）|
> | &nbsp;&nbsp;• `lora_rank` ≤ 64 |（rank 越高，占用显存越多）|
> | &nbsp;&nbsp;• `eval_steps` ≤ 20 |（频繁验证便于观察）|

**开始实验！**

MS-Swift 默认使用 **LoRA 微调**，无需额外指定。

此外，框架自带**学习率调度器**，会在训练过程中逐步降低学习率，避免越过最优解。这在初始学习率较高（如 `0.1`）时尤为重要，有助于稳定收敛。


In [ ]:
%env TF_ENABLE_ONEDNN_OPTS=0
%env CUDA_VISIBLE_DEVICES=0
%env LOG_LEVEL=INFO
!swift sft \
--learning_rate '0.1' \
--lora_rank 4 \
--num_train_epochs 1 \
--dataset './resources/training_dataset_100.jsonl' \
--per_device_train_batch_size 8 \
--eval_steps 1 \
--max_length 512 \
--model_type 'qwen2' \
--model './model/qwen2_5-1_5b-instruct'

|  训练损失  |  验证损失  |
|-----------------|-------------------|
|<img src="images/LMP-C01_08-Exp1TL.png" style="max-width: 500px"/> | <img src="images/LMP-C01_08-Exp1EL.png" style="max-width: 500px"/> |

|  **观察**  |  训练与验证损失双双上扬  |
| --- | --- |
|  **训练状态**  |  **失败**  |
|  **分析**  |  学习率过高，优化器不断越过最优点，无法收敛。<br><img src="images/LMP-C01_08-Exp1Overshoot.png" style="max-width: 300px"> |
|  **调整策略**  |  将学习率大幅降低至 `0.00005`。  |

### 实验 2（≈5 分钟）
    
由于第一次实验越过了收敛点，这次我们将学习率显著调低：

> **参数设置**
> 
> | 参数 | 原始值 | 新值  |
> |--------|-------|-------|
> | **学习率** (`learning_rate`) | `0.1` | `0.00005` |
> | **LoRA 秩** (`lora_rank`) | `4` |  |
> | **训练轮数** (`num_train_epochs`) | `50` |  |
> | **数据集路径** | `"./resources/training_dataset_100.jsonl"` |  |
> | **批量大小** (`batch_size`) | `8` |  |
> | **验证步数** (`eval_steps`) | `1` |  |

In [ ]:
%env TF_ENABLE_ONEDNN_OPTS=0
%env CUDA_VISIBLE_DEVICES=0
%env LOG_LEVEL=INFO
!swift sft \
--learning_rate '0.00005' \
--lora_rank 4 \
--num_train_epochs 1 \
--dataset './resources/training_dataset_100.jsonl' \
--per_device_train_batch_size 8 \
--eval_steps 1 \
--max_length 512 \
--model_type 'qwen2' \
--model './model/qwen2_5-1_5b-instruct'

|  训练损失  |  验证损失  |
|-----------------|-------------------|
|<img src="images/LMP-C01_08-Exp2TL.png" style="max-width: 500px"/> | <img src="images/LMP-C01_08-Exp2EL.png" style="max-width: 500px"/> |

|  **观察**  |  训练损失与验证损失均呈下降趋势  |
| --- | --- |
|  **训练状态**  |  **欠拟合**  |
|  **分析**  |  欠拟合是训练中非常常见的问题，说明模型还没有充分学到数据中的模式。以当前参数继续训练更久有望收敛，也可以尝试调整超参数以加速训练。 |
|  **调整策略**  |  1. 延长训练时间：将 epoch 提升至 `50`。<br>2. 将 `batch_size` 增加到显存允许的最大值 `16`，以加快训练。  |

### 实验 3（≈10 分钟）
    
基于上一轮的结果，我们继续调整：

> **参数设置**
> 
> | 参数 | 原始值 | 新值  |
> |--------|-------|-------|
> | **学习率** (`learning_rate`) | `0.00005` |  |
> | **LoRA 秩** (`lora_rank`) | `4` |  |
> | **训练轮数** (`num_train_epochs`) | `1` | `50` |
> | **数据集路径** | `"./resources/training_dataset_100.jsonl"` |  |
> | **批量大小** (`batch_size`) | `8` | `16` |
> | **验证步数** (`eval_steps`) | `1` | `20` |

In [ ]:
%env TF_ENABLE_ONEDNN_OPTS=0
%env CUDA_VISIBLE_DEVICES=0
%env LOG_LEVEL=INFO
!swift sft \
--learning_rate '0.00005' \
--lora_rank 4 \
--num_train_epochs 50 \
--dataset './resources/training_dataset_100.jsonl' \
--per_device_train_batch_size 16 \
--eval_steps 20 \
--max_length 512 \
--model_type 'qwen2' \
--model './model/qwen2_5-1_5b-instruct' 

|  训练损失  |  验证损失  |
|-----------------|-------------------|
|<img src="images/LMP-C01_08-Exp3TL.png" style="max-width: 500px"/> | <img src="images/LMP-C01_08-Exp3EL.png" style="max-width: 500px"/> |

|  **观察**  |  训练损失持续下降，但验证损失先降后升  |
| --- | --- |
|  **训练状态**  |  **过拟合**  |
|  **分析**  |  过拟合是训练中常见的现象，说明模型开始“背答案”，而不是学习数据中的规律。我们需要减少训练轮数或增加训练数据，帮助模型从记忆走向泛化。 |
|  **调整策略**  |  1. 将训练轮数减至 `3`。<br>2. 将训练样本提升至 `500` 份，数据位于 `./resources/training_dataset_500.jsonl`。<br>3. 数据量增加后，把 LoRA 秩提高到 `16`。  |

### 实验 4（≈5 分钟）
    
针对过拟合，我们同时进行以下三项调整：
* 减少训练轮数
* 增加训练样本
* 提升 LoRA 秩

> **参数设置**
> 
> | 参数 | 原始值 | 新值  |
> |--------|-------|-------|
> | **学习率** (`learning_rate`) | `0.00005` |  |
> | **LoRA 秩** (`lora_rank`) | `4` | `8` |
> | **训练轮数** (`num_train_epochs`) | `50` | `3` |
> | **数据集路径** | `"./resources/training_dataset_100.jsonl"` | `"./resources/training_dataset_500.jsonl"` |
> | **批量大小** (`batch_size`) | `16` |  |
> | **验证步数** (`eval_steps`) | `20` |  |

In [ ]:
%env TF_ENABLE_ONEDNN_OPTS=0
%env CUDA_VISIBLE_DEVICES=0
%env LOG_LEVEL=INFO
!swift sft \
--learning_rate '0.00005' \
--lora_rank 8 \
--num_train_epochs 3 \
--dataset './resources/training_dataset_500.jsonl' \
--per_device_train_batch_size 16 \
--eval_steps 20 \
--max_length 512 \
--model_type 'qwen2' \
--model './model/qwen2_5-1_5b-instruct' 

|  训练损失  |  验证损失  |
|-----------------|-------------------|
|<img src="images/LMP-C01_08-Exp4TL.png" style="max-width: 500px"/> | <img src="images/LMP-C01_08-Exp4EL.png" style="max-width: 500px"/> |

|  **观察**  |  训练与验证损失都在底部平台区波动  |
| --- | --- |
|  **训练状态**  |  **欠拟合**  |
|  **分析**  |  我们可能已经非常接近成功收敛！ |
|  **调整策略**  |  将训练轮数增加到 `50`。 |

### 实验 5（≈20 分钟）

看起来已经接近收敛，但我们需要确认。于是我们延长训练时间，观察会发生什么：
    
> **参数设置**
> 
> | 参数 | 原始值 | 新值  |
> |--------|-------|-------|
> | **学习率** (`learning_rate`) | `0.00005` |  |
> | **LoRA 秩** (`lora_rank`) | `8` |  |
> | **训练轮数** (`num_train_epochs`) | `3` | `50` |
> | **数据集路径** | `"./resources/training_dataset_100.jsonl"` | `"./resources/training_dataset_1000.jsonl"` |
> | **批量大小** (`batch_size`) | `16` |  |
> | **验证步数** (`eval_steps`) | `20` |  |

In [ ]:
%env TF_ENABLE_ONEDNN_OPTS=0
%env CUDA_VISIBLE_DEVICES=0
%env LOG_LEVEL=INFO
!swift sft \
--learning_rate '0.00005' \
--lora_rank 8 \
--num_train_epochs 50 \
--dataset './resources/training_dataset_500.jsonl' \
--per_device_train_batch_size 16 \
--eval_steps 20 \
--max_length 512 \
--model_type 'qwen2' \
--model './model/qwen2_5-1_5b-instruct' 

|  训练损失  |  验证损失  |
|-----------------|-------------------|
|<img src="images/LMP-C01_08-Exp5TL.png" style="max-width: 500px"/> | <img src="images/LMP-C01_08-Exp5EL.png" style="max-width: 500px"/> |

|  **观察**  |  训练和验证损失都没有继续下降的迹象  |
| --- | --- |
|  **训练状态**  |  **成功！**  |

此次实验的曲线与**实验 4** 高度一致，进一步印证我们已经达到收敛！

## 重新进行基线测试

当我们使用 `ms-swift` 成功微调 LLM 后，会生成一系列检查点文件。

通常我们会选用最新的检查点作为最终 LoRA。你可以在 `/output/` 目录下找到这些检查点：


<div style="text-align: center;">
  <img src="images/LMP-C01_08-Checkpoints.gif" style="max-width: 800px;" />
  <br>
  <small>如何找到最新的 LoRA 检查点</small>
</div>


现在让我们来验证刚刚微调完成的 LLM：

In [ ]:
from swift.tuners import Swift

# Replace ckpt_dir with the directory pointing to your checkpoint
--ckpt_dir 'output/qwen2_5-1_5b-instruct/<replace_with_checkpoint_directory>' \ # ← Replace with checkpoint directory

# Load the model
ft_model = Swift.from_pretrained(model, ckpt_dir, inference_mode=True)

In [ ]:
import json
from IPython.display import Markdown

sum, score = 0, 0
for line in open("./resources/benchmark_exam.jsonl"):

    math_question = json.loads(line)
    query = math_question["messages"][1]["content"]

    response, _ = inference(model, template, query)

    ans = math_question["messages"][2]["content"]
    pos = ans.find("ans")
    end_pos = ans[pos:].find('}}')
    ans = ans[pos - 2: end_pos + pos + 2]
    
    print("=" * 50)
    print(query.split("#Math Problem#\n")[1])
    print("The answer we're looking for is: " + ans + "\n")
    print("-" * 25 + "Model response" + "-" * 25)
    display(Latex(response)) 
    print("-" * 23 + "Model response end" + "-" * 23)

    if ans in response or ans[6:-2] in response:
        score += 1
        print("Model is correct!\n")
    else: print("Model is wrong :(\n")
    sum += 1

display(Markdown("Benchmark score: **" + str(int(100*score/sum)) + "**"))

## 模型合并

微调完成后，我们可以将模型投入使用，主要有两种方式：

* **在加载基线模型时加载 LoRA**  
    LoRA 在微调过程中生成的低秩矩阵通常非常小——大约只有 20MB 左右。这种轻量特性非常适合增量更新与分发，是工程场景广泛采用的方式。需要注意的是，推理时必须确保 LoRA 适配器与原始基线模型完全匹配，也就是说推理时指定的基线模型必须与训练时一致，才能正确应用微调参数。  
    前一小节我们已经演示过这种做法：通过 `ckpt_dir` 参数加载适配器权重。<br><br>
* **将 LoRA 权重合并回基线模型**  
    另一种做法是直接把微调得到的低秩参数永久融入原始模型，得到一个全新的独立模型。合并后的模型不再依赖外部适配器文件，可以像常规模型一样单独部署。

本节我们简单展示第二种方式：模型合并。其本质是将 LoRA 中的权重矩阵合并到基线模型的权重里。

`swift export` 工具让合并过程非常简单。只需提供微调后的检查点路径——最好是训练过程中验证损失最低的 `best_model_checkpoint` ——工具就会输出一个可直接部署的完整模型。

> **专业提示：**  
> 如果 `ms-swift` 输出了 `best_model_checkpoint`，强烈建议优先使用它。

In [ ]:
%env LOG_LEVEL=INFO
!swift export \
--ckpt_dir 'output/qwen2_5-1_5b-instruct/<replace_with_checkpoint_directory>' \ # ← Replace with checkpoint directory
--merge_lora true

<div style="text-align: center;">
  <img src="images/LMP-C01_08-Model Merging.jpg" style="max-width: 800px; border: 1px solid white;" />
  <br>
  <small>合并模型的存储位置</small>
</div>

从日志中可以看到模型已成功合并，并保存到了一个新文件夹：

```python
"/output/qwen2_5-1_5b-instruct/v##-yyyymmdd-hhmmss/checkpoint-#####-merged"
```

# 接下来做什么？

---

## 自测一下！
<details>
<summary style="cursor: pointer; padding: 12px; border: 1px solid #dee2e6; border-radius: 6px;">
<b>1. 使用 LoRA（低秩适配）微调大语言模型的主要优势是什么？</b>

<ul>
<li>A) 显著增加模型的参数量</li>
<li>B) 在保持较低内存占用的情况下重训全部模型权重</li>
<li>C) 将可训练参数减少最高可达 98%，在维持性能的同时降低成本</li>
<li>D) 让模型完全不需要训练数据</li>
</ul>

查看答案 →

</summary>

<div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **正确答案：** C) 将可训练参数减少最高可达 98%，在维持性能的同时降低成本  
📝 **说明：**  
LoRA 会冻结原始模型权重，仅通过低秩矩阵（A 和 B）表示权重更新。这样一来，只有极少量参数需要训练——显著降低显存和训练成本，同时能得到接近全量微调的效果。

</div>
</details>

<br>

<details>
<summary style="cursor: pointer; padding: 12px; border: 1px solid #dee2e6; border-radius: 6px;">
<b>2. 如果训练损失下降而验证损失上升，意味着什么？</b>

<ul>
<li>A) 模型欠拟合</li>
<li>B) 模型泛化良好</li>
<li>C) 模型过拟合</li>
<li>D) 学习率太低</li>
</ul>

查看答案 →

</summary>

<div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **正确答案：** C) 模型过拟合  
📝 **说明：**  
这种模式表明模型在训练数据上表现越来越好，但在验证数据上表现变差。说明模型正在“死记硬背”训练集，而不是掌握可泛化的规律。这就是“记答案陷阱”。

</div>
</details>

<br>

<details>
<summary style="cursor: pointer; padding: 12px; border: 1px solid #dee2e6; border-radius: 6px;">
<b>3. 微调过程中，验证集的主要作用是什么？</b>

<ul>
<li>A) 用于训练模型参数</li>
<li>B) 用于训练结束后的最终评估</li>
<li>C) 用于监控训练进度并防止过拟合</li>
<li>D) 用于扩大批量大小</li>
</ul>

查看答案 →

</summary>

<div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **正确答案：** C) 用于监控训练进度并防止过拟合  
📝 **说明：**  
验证集好比“模拟考试”，可以在训练过程中评估模型在未见过数据上的表现。通过观察验证损失，我们可以尽早发现过拟合并采取措施，例如提前停止训练。

</div>
</details>

<br>

## 核心要点

* **为什么要微调**
    * **提示工程有天花板**——提示词再精妙，也无法弥补逻辑和推理能力的不足。
    * **微调教会模型如何思考**，而不仅仅是“说得好听”。它能明显提升逻辑、数学和领域专用推理能力。
    * **混合策略是制胜之道**：用微调提升推理能力，搭配工具或插件确保准确性。例如，让模型掌握解题步骤，再用计算器完成精确运算。

<br>

* **预训练 vs. 微调**
    * **预训练：** 从海量未标注数据（如书籍、网页）中学习通用语言能力。
    * **微调：** 使用少量标注数据（如数学题、客服语料）把预训练模型定制到具体任务。
    * 无需从零开始训练——复用 Qwen、Llama、DeepSeek 等强大开源模型即可。

<br>

* **参数高效微调（PEFT）**
    * 全量微调需要更新所有参数——**显存和成本都很高**。
    * PEFT 方法（如 LoRA）只更新少量参数——**开销极低、部署灵活**。
    * **LoRA 成为事实标准：**
        * 冻结原始权重。
        * 通过低秩矩阵 ΔW=A⋅B 表示增量。
        * 将可训练参数从百万级降至千级。
        * 以约 1–2% 的成本获得接近全量微调的效果。